**Task 07: Querying RDF(s)**

In [ ]:
!pip install rdflib 
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4"

     |████████████████████████████████| 235kB 2.8MB/s 
     |████████████████████████████████| 51kB 5.1MB/s 


Leemos el fichero RDF de la forma que lo hemos venido haciendo

In [ ]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/resources/example6.rdf", format="xml")

<Graph identifier=Nba85461781974efd9b44ed49289e37f7 (<class 'rdflib.graph.Graph'>)>

**TASK 7.1: List all subclasses of "Person" with RDFLib and SPARQL**

In [ ]:
ns = Namespace("http://somewhere#")
print("RDFLib:")
for s,p,o in g.triples((None, RDFS.subClassOf, ns.Person)):
  print(s,p,o)

RDFLib:
http://somewhere#Researcher http://www.w3.org/2000/01/rdf-schema#subClassOf http://somewhere#Person


In [ ]:
from rdflib.plugins.sparql import prepareQuery

q1 = prepareQuery('''
  SELECT ?s
  WHERE { 
    ?s rdfs:subClassOf ns:Person. 
  }
  ''',
  initNs = { "rdfs": RDFS, "ns": ns}
)
print("SPARQL:")
for s in g.query(q1):
  print(s)

SPARQL:
(rdflib.term.URIRef('http://somewhere#Researcher'),)


**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


In [ ]:
print("RDFLib:")
for s,p,o in g.triples((None, RDF.type, ns.Person)):
  print(s)
for s,p,o in g.triples((None, RDFS.subClassOf, ns.Person)):
  for ss,pp,oo in g.triples((None, RDF.type, s)):
    print(ss)

RDFLib:
http://somewhere#JohnSmith
http://somewhere#SaraJones
http://somewhere#JaneSmith


In [ ]:
q2 = prepareQuery('''
  SELECT ?ss
  WHERE { 
    { 
      ?ss rdf:type ns:Person. 
    } UNION {
      ?sss rdfs:subClassOf ns:Person.
      ?ss rdf:type ?sss
    }
  }
  ''',
  initNs = { "rdfs": RDFS,"rdf": RDF, "ns": ns}
)
print("SPARQL:")
for s2 in g.query(q2):
  print(s2)

SPARQL:
(rdflib.term.URIRef('http://somewhere#JohnSmith'),)
(rdflib.term.URIRef('http://somewhere#SaraJones'),)
(rdflib.term.URIRef('http://somewhere#JaneSmith'),)


**TASK 7.3: List all individuals of "Person" and all their properties including their class with RDFLib and SPARQL**


In [ ]:
print("RDFLib:")
for s,p,o in g.triples((None, RDF.type, ns.Person)):
  for ss,pp,oo in g.triples((s, None, None)):
      print(ss,pp,oo)

for s,p,o in g.triples((None, RDFS.subClassOf, ns.Person)):
  for ss,pp,oo in g.triples((None, RDF.type, s)):
    for sss,ppp,ooo in g.triples((ss, None, None)):
      print(sss,ppp,ooo)

RDFLib:
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN John Smith
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given John
http://somewhere#JohnSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Smith
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Sara
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Jones
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN Sara Jones
http://somewhere#SaraJones http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Jane
http://somewhere#JaneSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Researcher
http://somewhere#JaneSmith http://som

In [ ]:
q3 = prepareQuery('''
  SELECT ?s ?p ?o
  WHERE { 
    { 
      ?s rdf:type ns:Person.
      ?s ?p ?o
    } UNION {
      ?s rdf:type ?ss.
      ?s ?p ?o.
      ?ss rdfs:subClassOf ns:Person
    }
  }
  ''',
  initNs = { "rdfs": RDFS,"rdf": RDF, "ns": ns}
)
print("SPARQL:")
for s3 in g.query(q3):
  print(s3)

SPARQL:
(rdflib.term.URIRef('http://somewhere#JohnSmith'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN'), rdflib.term.Literal('John Smith', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://somewhere#JohnSmith'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given'), rdflib.term.Literal('John', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://somewhere#JohnSmith'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://somewhere#Person'))
(rdflib.term.URIRef('http://somewhere#JohnSmith'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family'), rdflib.term.Literal('Smith', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://somewhere#SaraJones'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vc